In [ ]:
!nvidia-smi

Thu Mar 14 09:03:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/DACON 경진대회/한솔 도배하자 질의응답/최종 제출/data/'
SEED = 42
reset_seeds(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# model load

In [ ]:
from transformers import GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

# inference function

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score
def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
def post_processing(text):
    import re
    end = text.rfind('</s>')
    if end:
        text = text[:end]
    text = text.strip()
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

# 실제 테스트

In [ ]:
test = pd.read_csv(f'{DATA_PATH}test.csv')
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [ ]:
def Chatbot(input_text, model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7, num_samples=5, generated=True):

    model.eval()
    input_text = input_text.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    score_lst = []
    generated_texts = []
    for i in range(num_samples):
        result_ids = model.generate(input_ids,
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    do_sample=True,
                                    num_return_sequences=1,
                                    )

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = post_processing(generated_text[q_len:])

        similarity_score = evaluate_similarity(input_text, generated_text)
        score_lst.append(similarity_score)
        generated_texts.append((similarity_score,generated_text))
        if generated:
            print(generated_texts[i])

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    return best_generated_text, score_lst

In [ ]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(test.iterrows(), total=test.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    answer, score = Chatbot(text, model=model, tokenizer=tokenizer, generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)
    answer_list.append(answer)
    mean_score.append(score)

  0%|          | 0/130 [00:00<?, ?it/s]

In [ ]:
for a, b, c in zip(test['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 실크벽지와 페인트가 다릅니다. 실크벽지는 오염이 잘 생기고 저렴해보인다는 장점이 있지만 수분을 많이 흡수하고 변색이 있습니다.
답변 : [0.2246882709173056, 0.19993112564086912, 0.2246882709173056, 0.2246882709173056, 0.3364462884572836]
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 주로 녹 오염으로 인해 발생하는 경우가 많습니다. 이러한 경우에는 도배지 주변의 금속이나 화학물질로 인한 녹이 도배지에 베어나올 수 있습니다. 따라서 빠른 시일 내에 누수를 막고, 도배지를 건조시키는 것이 중요합니다. 또한, 누수로 인한 녹이 도배지에 베어나올 수 있는 경우에는 전문가의 도움을 받아 누수 부분을 처리하고 곰팡이가 발생한 부분을 보수하는 것도 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.
답변 : [0.3870775442231785, 0.39559360218770573, 0.42450813165417417, 0.415729654377157, 0.3745015595332686]
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법 단열판을 사용하는 것의 장점은 습기 억제 능력이 뛰어나고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 점입니다.
답변 : [0.5323848014528101, 0.5

# submission

In [ ]:
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
pred_vector = vector_model.encode(answer_list)
pred_vector.shape

(130, 512)

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission.iloc[:,1:] = pred_vector
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.001564,0.010433,0.011153,0.017342,0.074837,0.000671,0.043165,-0.042092,-0.014254,...,-0.000056,-0.038679,0.007388,-0.058834,0.010555,-0.000304,0.056837,0.029294,0.012208,0.027119
1,TEST_001,-0.041846,-0.007950,0.032942,0.006954,0.103263,0.009337,-0.012238,-0.033785,-0.026159,...,-0.033484,-0.034196,0.011026,-0.046404,-0.000631,0.029510,0.005405,-0.059029,0.018402,0.002521
2,TEST_002,0.026752,-0.051112,-0.031208,-0.008895,0.123851,-0.055075,-0.024506,-0.026343,0.036229,...,-0.041047,-0.010087,0.032355,-0.020387,0.011134,0.008836,-0.038128,-0.040148,-0.050396,0.094210
3,TEST_003,0.020456,0.014602,-0.005448,0.021697,0.066295,-0.049532,-0.060348,-0.018360,-0.011505,...,-0.044512,0.017548,0.052905,-0.040040,0.010618,0.013271,-0.044331,-0.028638,0.021200,0.021273
4,TEST_004,-0.019904,0.088827,0.025052,-0.010172,-0.030689,0.080919,0.063261,0.061294,0.024599,...,-0.005135,-0.012875,0.116320,0.050086,0.030426,0.026419,-0.065965,-0.042933,0.015363,0.020562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.030861,-0.044582,0.027274,0.015991,0.117427,-0.067691,0.066928,0.042426,0.054038,...,0.015964,0.010538,0.047512,-0.039530,0.021517,0.048241,0.044327,-0.024992,-0.032482,0.007421
126,TEST_126,0.048317,-0.062851,-0.024441,0.005181,0.099421,-0.031031,0.020363,0.002396,0.018256,...,-0.038585,-0.035280,0.006957,-0.025356,0.016975,0.027192,-0.009129,-0.034732,-0.011866,0.060189
127,TEST_127,-0.032080,0.006587,0.020201,-0.000382,0.090871,-0.010097,-0.011154,-0.042685,-0.009931,...,-0.026458,-0.020740,-0.005004,-0.018524,-0.014665,0.028693,0.004136,0.023824,-0.058017,0.053458
128,TEST_128,0.036386,-0.027929,-0.063045,0.015607,0.084918,-0.076239,0.011936,0.024018,-0.014002,...,-0.026435,-0.025367,0.003164,-0.060882,-0.018098,0.008412,0.012525,-0.015694,0.026506,0.010083


In [ ]:
submission.to_csv(f'{DATA_PATH}hansol_120000_3epoch_rule_based_AdamW_lr_scheduler.csv', index=False)